# 1. Многоклассовая классификация

In [14]:
import numpy as np
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 3000
X_test /= 3000
X_train = np.hstack((X_train, np.ones((len(X_train), 1))))
X_test = np.hstack((X_test, np.ones((len(X_test), 1))))

In [17]:
def soft_max(v):
    s = np.sum(np.exp(v))
    ans = np.zeros(len(v))
    for i in range(len(v)):
        ans[i] = np.exp(v[i]) / s
    return ans

In [122]:
def gradient(p, x, label):
        grad = np.dot(np.split(p, 10), np.array([x]))
        grad[label] -= x
        return grad

In [27]:
class Classifier:
    def __init__(self):
        self.w = np.random.uniform(0, 1, (10, 785))

    def learn(self, x, y):
        N = len(x)
        for i in range(N):
            p = soft_max(np.dot(self.w, x[i]))
            self.w -= gradient(p, x[i], y[i])

    def classify(self, x):
        p = soft_max(np.dot(self.w, x))
        ans = 0
        for i in range(10):
            if p[i] > p[ans]:
                ans = i
        return round(ans)

    def measure_accuracy(self, X_test, y_test):
        cnt = 0.
        for i in range(len(X_test)):
            answer = self.classify(X_test[i])
            if answer == y_test[i]:
                cnt += 1
        return cnt / len(X_test)

In [123]:
cl = Classifier()
cl.learn(X_train, y_train)
print(cl.measure_accuracy(X_test, y_test))

0.9093


 ## Регуляризация

In [136]:
k1, k2 = 2e-5, 2e-2

def gradient(p, x, label):
        grad = np.dot(np.split(p, 10), np.array([x]))
        grad[label] -= x
        grad += np.sign(grad) * k1 + k2 * grad
        return grad

In [138]:
cl = Classifier()
cl.learn(X_train, y_train)
print(cl.measure_accuracy(X_test, y_test))

0.9091


# 2. Полносвязные нейронные сети

#### 1. Предложить w и b, чтобы $y=\theta(wx+b)$ реализовывала следующие операторы:

$$1.~ wx + b = x_1\wedge x_2$$

$$2.~ wx + b = x_1\vee x_2$$

$$3.~ wx + b = \overline{x_1}\wedge \overline{x_2}$$

#### Решение

$$1.~ w = (1,1),~ b = -1.5$$

$$2.~ w = (1,1),~ b = -0.5$$

$$2.~ w = (-1,-1),~ b = 0.5$$

#### 2. Пример нереализуемой функции:
$$y = x_1 \equiv x_2$$


#### 3. Найти предствление через 

$$f(x)=\theta(W_2\theta(W_1x+b_1) + b_2)$$

#### Решение
$$y = (x_1 \wedge x_2) \vee (\overline{x_1}\wedge \overline{x_2})$$

$$W_1 = \begin{pmatrix}
1 & 1 \\
-1 & -1 \\
\end{pmatrix}$$

$$b_1=\begin{pmatrix}
-1.5 \\
0.5
\end{pmatrix}$$

$$W_2=(1,1),~ b_2=-0.5$$

Первый элемент вектора $\theta(W_1x+b_1)$ - значение левой скобки, второй - правой.

Оператор $\theta(W_2x+b)$ - дизъюнкция этих скобок.

#### 4. Представление произвольной булевой функции

Любая булева функция может быть представлена в ДНФ с помощью таблицы истинности. Остается реализовать конъюнкции.

Каждая строка матрицы $W_{1i}$ будет отвечать за $i$-ю скобку. В ней стоят единицы для литералов, которые взяты в скобке без отрицания, -1 для литералов с отрицанием и 0 для остальных. $b_{1i} = -k + 0.5$, где $k$ - количество литералов, взятых без отрицания. Легко проверить, что полученное значение $W_{1i}x+b_{1i}>0~\Leftrightarrow~$ все литералы в скобке равны 1.

Строка $W_2$ будет состоять из $n$ единиц (где $n$ - количество скобок), а $b_2=-0.5$.

Тогда $W_2x+b_2>0~\Leftrightarrow~ \exists i:x_i=1$, то есть, $\theta(W_2x+b)$ - дизъюнкция $x_1\ldots x_n$.